In [8]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
import folium
from geopy.geocoders import Nominatim
from IPython.display import display
from time import sleep

In [ ]:
#x = [airways, railways, roadways, waterways]
#W = 100  #weight
#E = [1.58, 0.0095, 0.18, 0.0035] #emissions in kg of co2
#C = [100, 8, 15, 5] # cost per unit per km
#D = [] # distance between node i to j through each mode
#T = [] # time taken fom node i to j through each mode

[   0,    0, 1150, 1740, 1750,    0, 1300,    0, 1300,    0,    0], # Delhi",
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Ahmedabad\n",
[1150,    0,    0,  840, 1040,    0,  620,    0, 1660,    0,    0], # Mumbai\n",
[1740,    0,  840,    0,  290,    0,  500,    0, 1470,    0,    0], # Bengaluru\n",
[1750,    0, 1040,  290,    0,    0,  520,    0, 1300,    0,    0], # Chennai\n",
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Kochi\n",
[1300,    0,  620,  500,  520,    0,    0,    0, 1180,    0,    0], # Hyderabad\n",
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Nagpur\n",
[1300,    0, 1660, 1470, 1300,    0, 1180,    0,    0,    0,    0], # Kolkata\n",
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Guwahati\n",
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]  # Kanpur\n",

In [2]:
nodes = ['Delhi', 'Ahmedabad', 'Mumbai', 'Bengaluru', 'Chennai',
         'Kochi', 'Hyderabad', 'Nagpur', 'Kolkata', 'Guwahati', 'Kanpur']
modes = ['airways', 'railways', 'roadways', 'waterways']

# Parameters
W = 100
inf = 10**6  # large number for constraints
E = {'airways': 1.58, 'railways': 0.0095, 'roadways': 0.18, 'waterways': 0.0035}  # kg CO2/unit km
C = {'airways': 100, 'railways': 8, 'roadways': 15, 'waterways': 5}
SPEED_KMPH = {'airways': 800, 'railways': 70, 'roadways': 50, 'waterways': 20}  # avg speeds

D_air = [
# Del   Ahd   Mum   Ben   Che   Koc   Hyd   Nag   Kol   Guw   Kan
[ inf,  inf, 1150, 1740, 1750,  inf, 1300,  inf, 1300,  inf,  inf], # Delhi
[ inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf], # Ahmedabad
[1150,  inf,  inf,  840, 1040,  inf,  620,  inf, 1660,  inf,  inf], # Mumbai
[1740,  inf,  840,  inf,  290,  inf,  500,  inf, 1470,  inf,  inf], # Bengaluru
[1750,  inf, 1040,  290,  inf,  inf,  520,  inf, 1300,  inf,  inf], # Chennai
[ inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf], # Kochi
[1300,  inf,  620,  500,  520,  inf,  inf,  inf, 1180,  inf,  inf], # Hyderabad
[ inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf], # Nagpur
[1300,  inf, 1660, 1470, 1300,  inf, 1180,  inf,  inf,  inf,  inf], # Kolkata
[ inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf], # Guwahati
[ inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf]  # Kanpur
]


D_rail = [
# Del   Ahd   Mum   Ben   Che   Koc   Hyd   Nag   Kol   Guw   Kan
[   0,  940, 1440, 2300, 2200, 3000, 1600, 1000, 1500, 2600,  500], # Delhi
[ 940,    0,  520, 1400, 1600, 1900, 1300, 1150, 2100, 2800,  950], # Ahmedabad
[1440,  520,    0, 1100, 1400, 1600,  750,  900, 2150, 3000, 1150], # Mumbai
[2300, 1400, 1100,    0,  360,  590,  600, 1100, 2050, 2800, 1400], # Bengaluru
[2200, 1600, 1400,  360,    0,  700,  700, 1400, 1700, 2800, 1500], # Chennai
[3000, 1900, 1600,  590,  700,    0,  800, 1700, 2200, 3400, 2000], # Kochi
[1600, 1300,  750,  600,  700,  800,    0,  550, 1600, 2200, 1200], # Hyderabad
[1000, 1150,  900, 1100, 1400, 1700,  550,    0, 1200, 2000,  720], # Nagpur
[1500, 2100, 2150, 2050, 1700, 2200, 1600, 1200,    0,  600,  950], # Kolkata
[2600, 2800, 3000, 2800, 2800, 3400, 2200, 2000,  600,    0, 1700], # Guwahati
[ 500,  950, 1150, 1400, 1500, 2000, 1200,  720,  950, 1700,    0]  # Kanpur
]

D_road = [
# Del   Ahd   Mum   Ben   Che   Koc   Hyd   Nag   Kol   Guw   Kan
[   0,  930, 1400, 2150, 2200, 2750, 1500,  950, 1500, 2300,  480], # Delhi
[ 930,    0,  530, 1400, 1600, 1900, 1300, 1150, 2100, 2800,  950], # Ahmedabad
[1400,  530,    0,  980, 1330, 1400,  710,  830, 2050, 2800, 1100], # Mumbai
[2150, 1400,  980,    0,  350,  510,  570, 1000, 1870, 2640, 1300], # Bengaluru
[2200, 1600, 1330,  350,    0,  690,  630, 1280, 1660, 2670, 1400], # Chennai
[2750, 1900, 1400,  510,  690,    0,  730, 1540, 2100, 3300, 1850], # Kochi
[1500, 1300,  710,  570,  630,  730,    0,  510, 1500, 2100, 1150], # Hyderabad
[ 950, 1150,  830, 1000, 1280, 1540,  510,    0, 1100, 1967,  700], # Nagpur
[1500, 2100, 2050, 1870, 1660, 2100, 1500, 1100,    0,  560,  900], # Kolkata
[2300, 2800, 2800, 2640, 2670, 3300, 2100, 1967,  560,    0, 1600], # Guwahati
[ 480,  950, 1100, 1300, 1400, 1850, 1150,  700,  900, 1600,    0]  # Kanpur
]

D_water = [
# Del   Ahd   Mum   Ben   Che   Koc   Hyd   Nag   Kol   Guw   Kan
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Delhi
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Ahmedabad
[   0,    0,    0,    0, 2700, 1100,    0,    0, 4000,    0,    0], # Mumbai
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Bengaluru
[   0,    0, 2700,    0,    0, 1600,    0,    0, 1400,    0,    0], # Chennai
[   0,    0, 1100,    0, 1600,    0,    0,    0, 2900,    0,    0], # Kochi
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Hyderabad
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Nagpur
[   0,    0, 4000,    0, 1400, 2900,    0,    0,    0,    0,    0], # Kolkata
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0], # Guwahati
[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]  # Kanpur
]


In [3]:
# =========================
# Build D and T dictionaries
# =========================
D = {}
T = {}
D_matrices = {'airways': D_air, 'railways': D_rail, 'roadways': D_road, 'waterways': D_water}
for mode in modes:
    matrix = D_matrices[mode]
    for i_idx, i in enumerate(nodes):
        for j_idx, j in enumerate(nodes):
            dist = matrix[i_idx][j_idx]
            if dist > 0:
                D[(i, j, mode)] = dist
                T[(i, j, mode)] = dist / SPEED_KMPH[mode]


In [4]:
# =========================
# Optimization model
# =========================
def optimize_objective(source, destination, weight, max_time_hours, obj_type, use_time_constraint=False):
    model = pyo.ConcreteModel()
    model.D_index = pyo.Set(initialize=D.keys(), dimen=3)
    model.C = pyo.Param(modes, initialize=C)
    model.E = pyo.Param(modes, initialize=E)
    model.D = pyo.Param(model.D_index, initialize=D)
    model.T = pyo.Param(model.D_index, initialize=T)
    model.x = pyo.Var(model.D_index, domain=pyo.NonNegativeReals)

    # Objective function
    if obj_type == 'cost':
        model.Obj = pyo.Objective(expr=sum(model.x[i,j,m]*model.C[m]*model.D[i,j,m]
                                           for (i,j,m) in model.D_index), sense=pyo.minimize)
    elif obj_type == 'emission':
        model.Obj = pyo.Objective(expr=sum(model.x[i,j,m]*model.E[m]*model.D[i,j,m]
                                           for (i,j,m) in model.D_index), sense=pyo.minimize)
    else:
        model.Obj = pyo.Objective(expr=sum(model.x[i,j,m]*model.T[i,j,m]
                                           for (i,j,m) in model.D_index), sense=pyo.minimize)

    # Flow balance constraints
    def flow_rule(model, n):
        infl = sum(model.x[i, n, m] for i in nodes for m in modes if (i, n, m) in model.D_index)
        outf = sum(model.x[n, j, m] for j in nodes for m in modes if (n, j, m) in model.D_index)
        if n == source:
            return outf - infl == weight
        elif n == destination:
            return infl - outf == weight
        else:
            return outf - infl == 0
    model.Flow = pyo.Constraint(nodes, rule=flow_rule)

    # Optional time constraint
    if use_time_constraint:
        model.TimeLimit = pyo.Constraint(expr=sum(model.x[i,j,m]*model.T[i,j,m]
                                                  for (i,j,m) in model.D_index) <= max_time_hours)

    solver = pyo.SolverFactory('gurobi')
    results = solver.solve(model, tee=False)

    if results.solver.termination_condition != pyo.TerminationCondition.optimal:
        print(f"No feasible solution for {obj_type}.")
        return []

    res = []
    for (i,j,m) in model.D_index:
        if model.x[i,j,m].value and model.x[i,j,m].value > 0:
            dist = D[(i,j,m)]
            res.append((i, j, m, dist, T[(i,j,m)], C[m]*dist, E[m]*dist))
    return res

In [14]:
def get_node_input(prompt, nodes):
    nodes_lower = {node.lower(): node for node in nodes}  # map lowercase -> original
    while True:
        val = input(prompt).strip()
        val_lower = val.lower()
        if val_lower in nodes_lower:
            return nodes_lower[val_lower]  # return original casing
        else:
            print(f"Invalid input. Please enter a valid node name (case-insensitive).")

def get_positive_number(prompt):
    while True:
        val = input(prompt)
        try:
            num = float(val)
            if num > 0:
                return num
            else:
                print("Please enter a positive number.")
        except:
            print("Invalid input. Please enter a valid number.")

# Get user inputs
source = get_node_input("Enter source city (case-sensitive): ", nodes)
destination = get_node_input("Enter destination city (case-sensitive): ", nodes)
weight = get_positive_number("Enter shipment weight (positive number): ")
max_time_hours = get_positive_number("Enter max allowed delay/time in hours (positive number): ")

print(f"\nInputs received:\nSource: {source}\nDestination: {destination}\nWeight: {weight}\nMax Time: {max_time_hours}\n")

# Run optimization for different objectives
for obj in ['cost', 'emission', 'time']:
    print(f"\n--- Optimizing for {obj} ---")
    route = optimize_objective(source, destination, weight, max_time_hours, obj, use_time_constraint=False)
    for leg in route:
        print(f"{leg[0]} → {leg[1]} via {leg[2]} | {leg[3]} km | {leg[4]:.1f} h | ₹{leg[5]:.2f} | {leg[6]:.2f} kgCO₂")



Inputs received:
Source: Kochi
Destination: Guwahati
Weight: 1000.0
Max Time: 240.0


--- Optimizing for cost ---
Kochi → Chennai via railways | 700 km | 10.0 h | ₹5600.00 | 6.65 kgCO₂
Kolkata → Guwahati via railways | 600 km | 8.6 h | ₹4800.00 | 5.70 kgCO₂
Chennai → Kolkata via waterways | 1400 km | 70.0 h | ₹7000.00 | 4.90 kgCO₂

--- Optimizing for emission ---
Kolkata → Guwahati via railways | 600 km | 8.6 h | ₹4800.00 | 5.70 kgCO₂
Kochi → Kolkata via waterways | 2900 km | 145.0 h | ₹14500.00 | 10.15 kgCO₂

--- Optimizing for time ---
Bengaluru → Kolkata via airways | 1470 km | 1.8 h | ₹147000.00 | 2322.60 kgCO₂
Kochi → Bengaluru via railways | 590 km | 8.4 h | ₹4720.00 | 5.60 kgCO₂
Kolkata → Guwahati via railways | 600 km | 8.6 h | ₹4800.00 | 5.70 kgCO₂


In [15]:
import webbrowser

def get_lat_lon(city_names):
    geolocator = Nominatim(user_agent="indian_map_app")
    locations = {}
    for city in city_names:
        try:
            location = geolocator.geocode(city + ", India")
            if location:
                locations[city] = (location.latitude, location.longitude)
            else:
                print(f"Warning: Could not find location for {city}")
        except Exception as e:
            print(f"Error getting location for {city}: {e}")
        sleep(1)
    return locations

def create_route_map(route, all_nodes, title):
    # Get all unique cities in route plus all nodes
    route_nodes = set()
    for leg in route:
        route_nodes.add(leg[0])
        route_nodes.add(leg[1])
    nodes_to_map = list(set(all_nodes).union(route_nodes))
    locations = get_lat_lon(nodes_to_map)
    
    # Center map on India
    india_center = [22.0, 79.0]
    m = folium.Map(location=india_center, zoom_start=5)
    
    # Add all nodes as red dots
    for city, (lat, lon) in locations.items():
        folium.CircleMarker(
            location=(lat, lon),
            radius=6,
            color='red',
            fill=True,
            fill_color='red',
            popup=city
        ).add_to(m)
    
    # Add route lines with mode and distance/time/cost in popup
    for leg in route:
        start, end, mode, dist, time_hr, cost, emission = leg
        if start in locations and end in locations:
            folium.PolyLine(
                locations=[locations[start], locations[end]],
                color='blue',
                weight=3,
                popup=f"{start} → {end}\nMode: {mode}\nDistance: {dist} km\nTime: {time_hr:.1f} h\nCost: ₹{cost:.2f}\nEmission: {emission:.2f} kgCO₂"
            ).add_to(m)

    return m

# Assuming you have your inputs and the function optimize_objective already defined...

maps = {}
for obj in ['cost', 'emission', 'time']:
    print(f"\n--- Optimizing for {obj} ---")
    route = optimize_objective(source, destination, weight, max_time_hours, obj, use_time_constraint=False)
    for leg in route:
        print(f"{leg[0]} → {leg[1]} via {leg[2]} | {leg[3]} km | {leg[4]:.1f} h | ₹{leg[5]:.2f} | {leg[6]:.2f} kgCO₂")

    m = create_route_map(route, nodes, title=f"Route optimized for {obj}")
    filename = f"route_map_{obj}.html"
    m.save(filename)
    maps[obj] = filename
    print(f"Map for {obj} saved as {filename}")

# Optionally, open all maps automatically in browser
for obj, filename in maps.items():
    webbrowser.open(filename)



--- Optimizing for cost ---
Kochi → Chennai via railways | 700 km | 10.0 h | ₹5600.00 | 6.65 kgCO₂
Kolkata → Guwahati via railways | 600 km | 8.6 h | ₹4800.00 | 5.70 kgCO₂
Chennai → Kolkata via waterways | 1400 km | 70.0 h | ₹7000.00 | 4.90 kgCO₂
Map for cost saved as route_map_cost.html

--- Optimizing for emission ---
Kolkata → Guwahati via railways | 600 km | 8.6 h | ₹4800.00 | 5.70 kgCO₂
Kochi → Kolkata via waterways | 2900 km | 145.0 h | ₹14500.00 | 10.15 kgCO₂
Map for emission saved as route_map_emission.html

--- Optimizing for time ---
Bengaluru → Kolkata via airways | 1470 km | 1.8 h | ₹147000.00 | 2322.60 kgCO₂
Kochi → Bengaluru via railways | 590 km | 8.4 h | ₹4720.00 | 5.60 kgCO₂
Kolkata → Guwahati via railways | 600 km | 8.6 h | ₹4800.00 | 5.70 kgCO₂
Map for time saved as route_map_time.html


Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.


[92309:92309:0100/000000.607225:ERROR:content/zygote/zygote_linux.cc:660] write: Broken pipe (32)
